In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook contains the two code for generating splinter and cirlce images "manually" - but automatically. By kind of "photoshopping" errors on good wafer images. 

## Helper Functions

In [ ]:
import glob
import random
from PIL import Image, ImageStat, ImageDraw

def get_random_background():
  filename = random.choice(glob.glob('/content/drive/MyDrive/WaferImages/good/*.tif'))
  background = Image.open(filename).convert('RGBA')

  if uniform(0, 1) > 0.5:
    background = background.transpose(Image.FLIP_LEFT_RIGHT)

  if uniform(0, 1) > 0.5:
    background = background.transpose(Image.FLIP_TOP_BOTTOM)

  return background

def get_random_error(image_path):
  filename = random.choice(glob.glob(str(image_path) + '/*.*'))
  foreground = Image.open(filename).convert('RGBA')

  return foreground

def random_resize(image):
  resize_factors = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
  resize_factors_p = [0.01, 0.01, 0.01, 0.1, 0.1, 0.1, 0.01, 0.01, 0.01, 0.01, 0.01]

  width, height = image.size
  random_resize_factor = choices(resize_factors, resize_factors_p)[0]
  image = image.resize((int(width*random_resize_factor),int(height*random_resize_factor)), Image.ANTIALIAS)

  return image

def random_blur(image):
  image_width, image_height = image.size

  if image_width * image_height < 1000:
    random_blur = randint(1, 2)
  else:
    random_blur = randint(2, 6)
  layer = Image.new('RGBA', (image_width + (4 * random_blur), image_height + (4 * random_blur)), (0, 0, 0, 0))

  layer_width, layer_height = layer.size
  centered_postition = ((layer_width - image_width) // 2, (layer_height - image_height) // 2)
  layer.paste(image, centered_postition)

  layer = layer.filter(ImageFilter.GaussianBlur(radius = random_blur))

  return layer


def random_blend(image):
  image_width, image_height = image.size

  layer = Image.new('RGBA', (image_width, image_height), (0,0,0,0))
  layer = Image.blend(layer, image, 0.9)

  return layer

def random_rotate(image):
  random_rotation = randint(0, 360)
  image = image.rotate(random_rotation, expand = 1, resample = 2)

  return image


def place_on(background, image):
  image_width, image_height = image.size
  background_width, background_height = background.size  

  random_width = randint(35, background_width - 35 - image_width)
  random_height = randint(35, background_height - 35 - image_height)

  background.paste(image, (random_width, random_height), image)

  return background

def check_visibility(image):
  image_stats = ImageStat.Stat(image)
  mean_brightness = image_stats.mean[3]

  image_width, image_height = image.size
  image_area = image_width * image_height

  return mean_brightness, image_area


## Circle Generator

This code block generated *n* numbers of circle images as defined. There are a lot of compositions possible, so tweaking around the paramters give different results.

In [ ]:
# Source: https://stackoverflow.com/questions/5324647/how-to-merge-a-transparent-png-image-with-another-image-using-pil

from PIL import Image, ImageShow, ImageFilter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import randint, uniform, choices
import shutil

def generate_circle_img(image_amount = 1):
 
  # Generates a list of length == image_amount while each number of the list represents the amount of circle error on each image
  number_of_errors = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
  number_of_errors_p = [9, 1, 1, 1, 1, 1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
  amount_errors = choices(number_of_errors, number_of_errors_p, k=image_amount)  

  for i, errors in enumerate(amount_errors):
    # Picks a random good wafer
    background = get_random_background()
    background_width, background_height = background.size
    error_was_placed = False

    for error in range(0, errors):
      error_visible = False
      threshold_counter = 0

      # Makes sure error is actually visible on wafer
      while error_visible == False:
        #Picks random error from the folder that holds the raw circle errors
        error = get_random_error('/content/drive/MyDrive/SampleGeneration/errors/circle/')
        # Resize Error based on set probabilities
        if errors == 1:
          resize_factors = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
          resize_factors_p = [0.01, 0.01, 0.01, 0.1, 0.1, 0.05, 0.01, 0.01, 0.01, 0.01, 0.01]
        else:
          resize_factors = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
          resize_factors_p = [2, 2, 1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.3, 0.3, 0.3, 0.1]
        random_resize_factor = choices(resize_factors, resize_factors_p)[0]

        # Blur circle dependend on resize_factor
        if random_resize_factor < 0.7:
          error = error.filter(ImageFilter.GaussianBlur(radius = randint(5, 7)))
        else:
          error = error.filter(ImageFilter.GaussianBlur(radius = randint(7, 13)))

        # Resize errors 
        width, height = error.size
        resized_error = error.resize((int(width*random_resize_factor),int(height*random_resize_factor)), Image.ANTIALIAS)

        # Randomly rotate errors
        rotated_error = random_rotate(resized_error)

        # Check if they are actually visible, especially because of blurring them
        error_brightness, error_area = check_visibility(rotated_error)
        if error_brightness > 35 and error_area > 1000 or threshold_counter >= 50:
          error_visible = True
          break
        threshold_counter += 1
        # Break if after 50 iterations still no randomization is found that leads to a visible circle
        if threshold_counter >= 50:
          break
        # Place Circle on Background if it was visible
        else:
          background = place_on(background, rotated_error)
          error_was_placed = True

    # Only if actually a circle was placed on the background it should be saved as a "circle" image obviously
    if error_was_placed == True:
      background = background.convert('L')
      plt.imshow(background)
      plt.show()
      # Uncomment this line if you actually want to save images somewhere.
      #background.save("/content/circle/" + str(i) + ".tif")

    
generate_circle_img(image_amount=447)




## Splinter Generator

The splinter generator actually works quite simlar to the circle generator. I just uses different parameters as - of course - these are needed to represent the splinter class in a better way.

In [ ]:
from PIL import Image, ImageShow, ImageFilter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import randint, uniform, choices
import numpy as np


def generate_splinter_img(image_amount):
 
  number_of_errors = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  number_of_errors_p = [16, 1, 1, 1, 1, 1, 1, 1, 1, 2]
  amount_errors = choices(number_of_errors, number_of_errors_p, k=image_amount)
  for i, errors in enumerate(amount_errors):
    background = get_random_background()
    background_width, background_height = background.size
    error_was_placed = False

    for splinter in range(0, errors):
      error = get_random_error('/content/drive/MyDrive/SampleGeneration/errors/splinter')
  
     
      # Makes sure error is actually visible on wafer
      error_visible = False
      threshold_counter = 0
      while error_visible == False:
        resized_error = random_resize(error)
        rotated_error = random_rotate(resized_error)
        randomized_error = random_blur(rotated_error)

        error_brightness, error_area = check_visibility(randomized_error)
        if error_brightness > 45 and error_area > 400 or threshold_counter >= 50:
          error_visible = True
          break
        threshold_counter += 1

      if threshold_counter >= 50:
        break
      else:
        background = place_on(background, randomized_error)
        error_was_placed = True
        
    if error_was_placed:
      background = background.convert('L')
      plt.imshow(background)
      plt.show()
      # Uncomment this line if you want to save the generated picture somewhere.
      # background.save("/content/splinter/" + str(i) + ".tif")

generate_splinter_img(image_amount=1000)

